# **Notebook 6: Transformación Avanzada y Persistencia de Parámetros 🛠️**

En este notebook daremos el paso final antes del entrenamiento. Prepararemos las variables numéricas para optimizar la convergencia de modelos lineales (Regresión Logística).

**Pasos Clave:**
1. **Transformación Box-Cox:** Para estabilizar la varianza y normalizar la distribución de `avg_glucose_level` y `bmi`.
2. **Serialización (JSON):** Guardaremos los valores de $\lambda$ (lambda) para replicar la transformación en el futuro.
3. **Estandarización:** Aplicaremos `StandardScaler` para que todas las variables tengan media 0 y desviación 1.
4. **Exportación Final:** Generaremos el dataset listo para entrar a la fase de Machine Learning.

In [6]:
import pandas as pd
import numpy as np
import json
from scipy import stats
from sklearn.preprocessing import StandardScaler

# 1. Carga de datos original
df = pd.read_csv('dataset/healthcare-dataset-stroke-transformed.csv')

# 2. Estructura para persistencia de parámetros
model_parameters = {
    "box_cox_lambdas": {},
    "box_cox_shifts": {},
    "scaler_params": {}
}

cols_to_transform = ['age', 'avg_glucose_level', 'bmi']

# --- FASE 1: TRANSFORMACIÓN BOX-COX CON BLINDAJE ---
print("--- Iniciando Transformación Box-Cox ---")
for col in cols_to_transform:
    # Verificamos si existen valores <= 0 para aplicar un SHIFT (Desplazamiento)
    min_val = df[col].min()
    
    if min_val <= 0:
        # Si hay valores no positivos, sumamos el valor necesario para que el mínimo sea 1
        shift = abs(min_val) + 1
        df[col] = df[col] + shift
        model_parameters["box_cox_shifts"][col] = shift
        print(f"⚠️ Variable '{col}': Aplicado shift de {shift:.4f} (detectado valor <= 0)")
    else:
        model_parameters["box_cox_shifts"][col] = 0.0

    # Aplicación de Box-Cox
    # La función devuelve (datos_transformados, valor_lambda)
    df[col], lmbda = stats.boxcox(df[col])
    model_parameters["box_cox_lambdas"][col] = lmbda
    print(f"✅ Variable '{col}': Lambda óptimo = {lmbda:.4f}")

# --- FASE 2: ESTANDARIZACIÓN (Z-SCORE) ---
print("\n--- Iniciando Estandarización ---")
scaler = StandardScaler()
df[cols_to_transform] = scaler.fit_transform(df[cols_to_transform])

# Guardamos los parámetros del Scaler para reconstruirlo en producción
model_parameters["scaler_params"] = {
    "mean": dict(zip(cols_to_transform, scaler.mean_)),
    "scale": dict(zip(cols_to_transform, scaler.scale_)) # escala = desviación estándar
}

# --- FASE 3: PERSISTENCIA Y EXPORTACIÓN ---
# Guardar parámetros en JSON
with open('dataset/preprocessing_params.json', 'w') as f:
    json.dump(model_parameters, f, indent=4)

# Guardar Dataset Final
df.to_csv('dataset/healthcare-dataset-stroke-final-scaled.csv', index=False)

print("\n" + "="*40)
print("PROCESO FINALIZADO CON ÉXITO")
print("Archivo generado: healthcare-dataset-stroke-final-scaled.csv")
print("Parámetros guardados: preprocessing_params.json")
print("="*40)

--- Iniciando Transformación Box-Cox ---
✅ Variable 'age': Lambda óptimo = 0.8361
✅ Variable 'avg_glucose_level': Lambda óptimo = -1.0574
✅ Variable 'bmi': Lambda óptimo = 0.0029

--- Iniciando Estandarización ---

PROCESO FINALIZADO CON ÉXITO
Archivo generado: healthcare-dataset-stroke-final-scaled.csv
Parámetros guardados: preprocessing_params.json
